# Base models Qwen3-06B and Gemini API
This notebook is a **baseline evaluation** to assess the performance of pre-trained models (`Qwen3-065B` and `gemini-2.0-flash-exp` API) on the real estate price prediction task. 

The notebook performs the following steps:

1. Load dataset from remote (Kaggle/Hugging Face).
2. Preprocess dataset.
3. Evaluate pre-trained `Qwen\Qwen3-0.6B` model using regression metrics.
4. Evaluate Gemini API using regression metrics.
5. Save results to compare with the **fine-tuned LoRa `Qwen3-0.6B` model**.

---
## Setup
---

### **Install Dependencies**

In [23]:
# !pip install -qU transformers wandb google-generativeai huggingface_hub[hf_xet]
!pip install -qU  json_repair

### **Import Dependencies**

In [24]:
import os
import torch
import json
import json_repair
import pandas as pd
from IPython.display import Markdown, JSON
from tqdm import tqdm

from transformers import AutoModelForCausalLM, AutoTokenizer
import google.generativeai as genai

from utils import logging_config, evalute_model, timeit
from fine_tuning_helpers import apply_prompt_template, decode_response

In [67]:
os.environ['LOGS'] = '/kaggle/working/logs'
os.environ['RESULTS'] = '/kaggle/working/results'

os.makedirs(os.environ['LOGS'], exist_ok=True )
os.makedirs(os.environ['RESULTS'], exist_ok=True )

LOGGER = logging_config(log_dir=os.environ['LOGS'])
LOGGER

<_Logger utils (INFO)>

### **Define Tokens and Authenticate**

In [39]:
# If using kaggle 
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
hf_token = user_secrets.get_secret("HUGGINGFACEHUB_API_TOKEN")
gemini_token = user_secrets.get_secret("GOOGLE_API_KEY")

# uncomment if using colab
# from google.colab import userdata
# hf_token = userdata.get("HUGGINGFACEHUB_API_TOKEN")
# gemini_token = userdata.get("GOOGLE_API_KEY")

In [40]:
from huggingface_hub import whoami, login

# !huggingface-cli login --token {mytoken} # another method
login(token = hf_token)
genai.configure(api_key=gemini_token)
JSON(whoami())

<IPython.core.display.JSON object>

In [ ]:
# uncomment if using colab
# import kagglehub
# kagglehub.login(validate_credentials=True)

---
## Load Dataset
---

### Read data from remote (Kaggle/Hugging Face)

**Download the dataset from Kaggle**

In [ ]:
# # Uncomment if using colab

# kagglehub.dataset_download('hebamo7amed/real-estate-data-for-llm-fine-tuning')
# tabular_data_path = f"{data_path}/tabular_data"
# text_data_path = f"{data_path}/text_data"
# text_data_path

**Read Text Datasets**

In [ ]:
# with open(f"{text_data_path}/text_train_data.jsonl", "r") as f:
#   train_data = json.load(f)

# with open(f"{text_data_path}/text_val_data.jsonl", "r") as f:
#   val_data = json.load(f)

# with open(f"{text_data_path}/sample_50.jsonl", "r") as f:
#   sample_data = json.load(f)

# print("Training data size = ", len(train_data))
# print("Validation data size = ", len(val_data))
# print("Sample data size = ", len(sample_data))

### **Load Dataset Sample from hugging Face Hub**

A data sample that was created from structured real estate data and uploaded to Hugging Face in the first notebook. It is formatted for instruction-based fine-tuning an LLM.

In [49]:
from datasets import load_dataset
dataset = load_dataset(
    path  ='heba1998/real-estate-data-sample-for-llm-fine-tuning'
)
dataset

README.md:   0%|          | 0.00/88.0 [00:00<?, ?B/s]

llm_train_data.jsonl:   0%|          | 0.00/5.54M [00:00<?, ?B/s]

llm_val_data.jsonl:   0%|          | 0.00/222k [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['system', 'instruction', 'input', 'output', 'history'],
        num_rows: 5000
    })
    validation: Dataset({
        features: ['system', 'instruction', 'input', 'output', 'history'],
        num_rows: 200
    })
})

In [68]:
# Convert data to list of jsons or ``jsonl``

val_data = [sample for sample in dataset['validation']]
house_price = lambda sample : json_repair.loads(sample['output'])['estimated_house_price']
true_labels = [ house_price(sample) for sample in dataset['validation']]

---
## Evaluate Responses from Base LM `Qwen3-0.6B-Instruct`
---

### **Helper Function to get responses for base model**

This function uses another function from the `fine_tuning_helpers.py` and `utils` utility scripts.

In [69]:
@timeit
def batch_generate(model, tokenizer, data, device):
    predictions = []
    tokens_history = []
    bar_format = '{l_bar}{bar}| {n_fmt}/{total_fmt} [{elapsed}<{remaining}]'
    
    for idx, sample in enumerate(tqdm(data, total=len(data),
                                 desc="Get response from pre-trained `Qwen3-0.6B` model",
                                 ncols=100, colour='green')):
        # 1. PREPROCESSING: 
        # build the system and user prompt with the model chat template
        prompt = apply_prompt_template(sample, output_str='instruction')

        # 2. TOKENIZATION: Tokenize the text prompt message
        inputs = tokenizer([prompt], return_tensors="pt").to(device)
        n_input_tokens = len(inputs.input_ids[0])
        LOGGER.info(f"\t>> Tokenized to {n_input_tokens} tokens")
        
        # 3. GENERATION: Generate response
        response_tokens_ids = model.generate(
            inputs=inputs.input_ids,
            attention_mask=inputs.attention_mask,
        )
        n_output_tokens = len(response_tokens_ids[0])

        # 4. POSTPROCESSING: Return only the output tokens and  exclude input ids and then clean response
        response_text = decode_response(response_tokens_ids, inputs.input_ids, tokenizer)
        response_dict = json_repair.loads(response_text)            
        house_price = response_dict["estimated_house_price"]
        predictions.append(int(house_price))
        
        # Store BAse model metadata for cost calculation
        tokens_history.append({
                'id': idx,
                'input_tokens': n_input_tokens, 
                'output_tokens': n_output_tokens,
                'total_tokens': n_input_tokens + n_output_tokens
            })
            
    return predictions, tokens_history

### **Load `Qwen3-0.6B` from Hugging Face**

In [11]:
model_id = "Qwen/Qwen3-0.6B"
device = "cuda:0" if torch.cuda.is_available() else "cpu"

# Load the model and tokenizer
tokenizer_qwen = AutoTokenizer.from_pretrained(model_id)
model_qwen = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=None).to(device)

tokenizer_config.json:   0%|          | 0.00/9.68k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/726 [00:00<?, ?B/s]

2025-05-09 11:41:33.445210: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746790893.732709      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746790893.808796      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/1.50G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

### **Generate Response using Pre-trained `Qwen3-0.6B`.**

Generate responses from the base model `Qwen3-0.6B` in the validation dataset.

In [12]:
# Make batch prediction or text to number genration 
qwen_preds, qwen_tokens_history = batch_generate(model=model_qwen, 
                                                  tokenizer=tokenizer_qwen, 
                                                  data = val_data,
                                                  device = device)

Get response from pre-trained `Qwen3-0.6B` model: 100%|███████████| 200/200 [19:57<00:00,  5.99s/it]


 Data completed in 19.96 minutes.


> Prediction for 200 sample takes 19.96 minutes using pretrained `Qwen3-0.6B`.

### **Evaluation metrics**

This function is implemented in the `utils.py` utility script.

In [211]:
print("Actual Price", true_labels[:10])
print("Predicted Prices", qwen_preds[:10])

Actual Price [2500000.0, 295000.0, 299900.0, 699000.0, 239000.0, 11000.0, 470000.0, 449000.0, 250000.0, 339000.0]
Predicted Prices [55000, 128000, 85000, 85000, 85000, 85000, 85000, 85000, 85000, 85000]


In [7]:
pretrained_qwen_metrics = evalute_model(true_labels, qwen_preds)
JSON(pretrained_qwen_metrics)

<IPython.core.display.JSON object>

> Model produce bad result and don't follow the output schema

> Model predict the same value each time `85000` or `8500` because it is bais towards the example of the schema.

In [30]:
qwen_results = pd.DataFrame(qwen_tokens_history)
qwen_results['y_actual'] = true_labels
qwen_results['y_pred'] = qwen_preds

qwen_results 

,id,input_tokens,output_tokens,total_tokens,y_actual,y_pred
0,0,244,264,508,2500000.0,55000
1,1,246,266,512,295000.0,128000
2,2,245,265,510,299900.0,85000
3,3,246,266,512,699000.0,85000
4,4,245,265,510,239000.0,85000
...,...,...,...,...,...,...
195,195,230,250,480,349000.0,85000
196,196,241,261,502,220000.0,210000
197,197,244,264,508,389000.0,85000
198,198,231,251,482,582990.0,85000


### **Save Pre-trained`Qwen3-0.6B` Results**

In [31]:
with open(f"{os.environ['RESULTS']}/pretrained_qwen_metrics.json", 'w') as json_file:
    json.dump(pretrained_qwen_metrics, json_file, indent=4)

qwen_results.to_csv(f"{os.environ['RESULTS']}/pretrained_qwen_results.csv", index=False)

---
## Evaluate Responses of Gemini Model
---

Evaluate the responses from Gemini API using regression metrics.

### **Helper Function to get responses from `genai` SDK API**

This function uses another function from the `fine_tuning_helpers.py` and `utils` utility scripts.

In [51]:
from fine_tuning_helpers import extract_house_price

@timeit
def batch_api_generate(model, data):
    llm_predictions = []
    tokens_history = []
    bar_format = '{l_bar}{bar}| {n_fmt}/{total_fmt} [{elapsed}<{remaining}]'
    
    for idx, sample in enumerate(tqdm(data, total=len(data),
                                 desc="Get response from pre-trained `Qwen3-0.6B` model",
                                 ncols=100, colour='green')):
        LOGGER.info("-"*50)
        LOGGER.info(f"Sample id {idx}")
        LOGGER.info("-"*50)
        # 1. PREPROCESSING: 
        # build the system and user prompt with the model chat template
        prompt= apply_prompt_template(sample)

        # 2. GENERATION: Generate response
        response = model.generate_content(prompt)
        text_response = response.candidates[0].content.parts[0].text

        # 4. POSTPROCESSING:  return only the output tokens and  exclude input ids
        LOGGER.info(f"Response text for sample id={idx} \n {text_response}" )
        
        # Clean response
        house_price = extract_house_price(text_response)
        LOGGER.info(f"\t>> Predicted price: {house_price}")
        llm_predictions.append(int(house_price))
        
        # Store metadata for expenses calculation
        tokens_history.append({
                'id': idx,
                'input_tokens': response.usage_metadata.prompt_token_count, 
                'output_tokens': response.usage_metadata.candidates_token_count,
                'total_tokens': response.usage_metadata.total_token_count
            })
            
        LOGGER.debug(f"Updated tokens history for {idx}")

    return llm_predictions, tokens_history

In [27]:
list(model.name for model in genai.list_models())

['models/chat-bison-001',
 'models/text-bison-001',
 'models/embedding-gecko-001',
 'models/gemini-1.0-pro-vision-latest',
 'models/gemini-pro-vision',
 'models/gemini-1.5-pro-latest',
 'models/gemini-1.5-pro-001',
 'models/gemini-1.5-pro-002',
 'models/gemini-1.5-pro',
 'models/gemini-1.5-flash-latest',
 'models/gemini-1.5-flash-001',
 'models/gemini-1.5-flash-001-tuning',
 'models/gemini-1.5-flash',
 'models/gemini-1.5-flash-002',
 'models/gemini-1.5-flash-8b',
 'models/gemini-1.5-flash-8b-001',
 'models/gemini-1.5-flash-8b-latest',
 'models/gemini-1.5-flash-8b-exp-0827',
 'models/gemini-1.5-flash-8b-exp-0924',
 'models/gemini-2.5-pro-exp-03-25',
 'models/gemini-2.5-pro-preview-03-25',
 'models/gemini-2.5-flash-preview-04-17',
 'models/gemini-2.5-flash-preview-04-17-thinking',
 'models/gemini-2.5-pro-preview-05-06',
 'models/gemini-2.0-flash-exp',
 'models/gemini-2.0-flash',
 'models/gemini-2.0-flash-001',
 'models/gemini-2.0-flash-lite-001',
 'models/gemini-2.0-flash-lite',
 'models

### **Generate Response using Gemini.**

Generate responses from `gemini-2.0-flash-exp` in the validation dataset.

In [52]:
import google.generativeai as genai
import numpy as np

Configs = genai.GenerationConfig(max_output_tokens=200)

model_id = "models/gemini-2.0-flash-exp"
gemini = genai.GenerativeModel(model_name=model_id,
                               generation_config=Configs)

gemini_preds, gemini_tokens_history = batch_api_generate(gemini, data = val_data)

Get response from pre-trained `Qwen3-0.6B` model: 100%|███████████| 200/200 [02:27<00:00,  1.36it/s]


 Data completed in 2.45 minutes.


### **Evaluation metrics**

This function is implemented in the `utils.py` utility script.

In [53]:
print("Actual Price", true_labels[:10])
print("Predicted Prices", gemini_preds[:10])

Actual Price [2500000.0, 295000.0, 299900.0, 699000.0, 239000.0, 11000.0, 470000.0, 449000.0, 250000.0, 339000.0]
Predicted Prices [-1, -1, 345000, -1, -1, -1, -1, -1, -1, -1]


> **`-1`** indecate to that gemini didn't produce the result.

In [54]:
gemini_metrics = evalute_model(true_labels, gemini_preds)

JSON(gemini_metrics)

<IPython.core.display.JSON object>

In [59]:
gemini_results = pd.DataFrame(gemini_tokens_history)
gemini_results['y_actual'] = true_labels
gemini_results['y_pred'] = gemini_preds

missed_prec = len(gemini_results[gemini_results['y_pred']==-1]) *100 /len(val_data)
print(f">>>>>> Gemini can't predict {missed_prec}% from the given data <<<<<<")

gemini_results.head(5)

>>>>>> Gemini can't predict 72.0% from the given data <<<<<<


,id,input_tokens,output_tokens,total_tokens,y_actual,y_pred
0,0,257,78,335,2500000.0,-1
1,1,258,77,335,295000.0,-1
2,2,256,104,360,299900.0,345000
3,3,258,78,336,699000.0,-1
4,4,258,78,336,239000.0,-1


### **Save Gemini Results**

In [71]:
with open(f"{os.environ['RESULTS']}/gemini_metrics.json", 'w') as json_file:
    json.dump(gemini_metrics, json_file, indent=4)

qwen_results.to_csv(f"{os.environ['RESULTS']}/gemini_results.csv", index=False)

### **Save Results**

In [72]:
import json

results_metadata = {
    "Description": "Here is the time taken to predict 200 samples for the base models",
    "Qwen Base (min)": 2.45,
    "Gemini (min)": 19.96
}


with open(f"{os.environ['RESULTS']}/results_metadata.json", 'w') as json_file:
    json.dump(results_metadata, json_file, indent=4)

In [73]:
# to be downolad for further comparasions
!zip /kaggle/working/results_200.zip /kaggle/working/results

updating: kaggle/working/results/ (stored 0%)
